In [1]:
from tams_pr2_guzheng.onset_to_path import OnsetToPath
from tams_pr2_guzheng.msg import (
    RunEpisodeAction,
    RunEpisodeGoal,
    RunEpisodeActionResult,
    RunEpisodeResult,
    ExecutePathAction,
    ExecutePathGoal
)
from music_perception.msg import NoteOnset
from tams_pr2_guzheng.utils import row_from_result, stitch_paths, run_params

from tams_pr2_guzheng.paths import RuckigPath

import copy

In [2]:
import rospy
rospy.init_node('jupyter_playground')
import tf2_ros
import tf2_geometry_msgs
import actionlib

tf= tf2_ros.Buffer()
tf_listener= tf2_ros.TransformListener(tf)

In [3]:
run_episode = actionlib.SimpleActionClient('run_episode', RunEpisodeAction)
run_episode.wait_for_server()

True

In [4]:
execute_path = actionlib.SimpleActionClient('pluck/execute_path', ExecutePathAction)
execute_path.wait_for_server()

True

In [5]:
o2p= OnsetToPath()

In [49]:
o2p.pluck_table

,note,max_vel_y,max_vel_z,max_acc_y,max_acc_z,max_jerk_y,max_jerk_z,pre_y,pre_z,post_y,post_z,keypoint_pos_y,keypoint_pos_z,keypoint_vel_y,keypoint_vel_z,string_position,loudness,detected_note,onset_cnt,onsets
0,d5,0.1,0.1,1,1.5,8,8,-0.015,0.015,0.01,0.02,0.000384,-0.000894,0.005152,0.020390,0.05,25.706879,D5,1,"{'confidence': 0.9698290229000001, 'duration':..."
1,e5,0.1,0.1,1,1.5,8,8,-0.015,0.015,0.01,0.02,0.003526,-0.001818,0.008596,0.023683,0.05,23.427002,E5,1,"{'confidence': 0.9356595278000001, 'duration':..."


In [19]:
example_note = 'e5'

In [10]:
# run real episode
params = RuckigPath.random(note = 'e4', direction= 1.0, string_position= 0.05)
run_params(run_episode, params)
result = run_episode.get_result()
row = row_from_result(result)
o2p.add_sample(row)

In [7]:
# or add stub pluck
g = RunEpisodeResult()
g.parameters = RuckigPath.random(note= example_note, string_position= 0.1).action_parameters
o = NoteOnset()
o.confidence = 1.0
o.loudness = 6.0
o.note = example_note
g.onsets.append(o)
row = row_from_result(g)
o2p.add_sample(row)

In [40]:
melody=['d5', 'e5', 'fis5', 'a5', 'd6']
melody=[n.upper().replace('IS', '♯') for n in melody]

In [41]:
paths= []
for n in melody:
    p = o2p.get_path(note=n, loudness= 3.0)
    approach_path = copy.deepcopy(p)
    approach_path.poses = approach_path.poses[0:1]
    approach_pose = copy.deepcopy(approach_path.poses[0])
    approach_pose.pose.position.z += 0.020
    approach_path.poses.insert(0, approach_pose)
    paths.append(approach_path)
    paths.append(p)

In [42]:
stitched_path = stitch_paths(paths, tf)

In [44]:
execute_path.send_goal(ExecutePathGoal(path= stitched_path, finger= 'ff'))
execute_path.wait_for_result()

True